<a href="https://colab.research.google.com/github/alferreiro/rBioTest/blob/main/rBioTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip list

Package                               Version
------------------------------------- -------------------
absl-py                               1.4.0
absolufy-imports                      0.3.1
accelerate                            1.10.0
aiofiles                              24.1.0
aiohappyeyeballs                      2.6.1
aiohttp                               3.12.15
aiosignal                             1.4.0
alabaster                             1.0.0
albucore                              0.0.24
albumentations                        2.0.8
ale-py                                0.11.2
altair                                5.5.0
annotated-types                       0.7.0
antlr4-python3-runtime                4.9.3
anyio                                 4.10.0
anywidget                             0.9.18
argon2-cffi                           25.1.0
argon2-cffi-bindings                  25.1.0
array_record                          0.7.2
arviz                                 0.22.0
astro

In [2]:
%%capture [--no-stderr]
!pip install transformers==4.41.2 safetensors==0.4.3 torch boto3 pandas awscli

Check for access to Rbio model weights and explore checkpoints

In [2]:
!aws s3 ls s3://czi-rbio/ --no-sign-request

                           PRE rbio1-EXP/
                           PRE rbio1-GO+EXP/
                           PRE rbio1-GO-C/
                           PRE rbio1-GO-F/
                           PRE rbio1-GO/
                           PRE rbio1-MLP/
                           PRE rbio1-TF+EXP/
                           PRE rbio1-TF+GO+EXP/
                           PRE rbio1-TF+GO+MLP+EXP/
                           PRE rbio1-TF+GO+MLP/
                           PRE rbio1-TF/


Explore check points for rbio1-TF+GO+EXP: Post-trained using: experimental data acting as a “hard verifier” on the task at hand; Transcriptformer foundation model as “soft verifier” using PMI scores; and Gene Ontology (GO-all: GO-P + GO-C + GO-F) knowledge base as a “soft verifier” for biological facts consistency.

In [3]:
!aws s3 ls s3://czi-rbio/rbio1-TF+GO+EXP/ --no-sign-request

                           PRE global_step100000/
2025-08-18 18:22:31          0 _SUCCESS
2025-08-18 18:22:32        605 added_tokens.json
2025-08-18 18:22:32        685 config.json
2025-08-18 18:22:32        243 generation_config.json
2025-08-18 18:25:17         17 latest
2025-08-18 18:25:18    1671853 merges.txt
2025-08-18 18:25:18 4957560304 model-00001-of-00002.safetensors
2025-08-18 18:25:18 1214366696 model-00002-of-00002.safetensors
2025-08-18 18:25:18      35581 model.safetensors.index.json
2025-08-18 18:25:19      16389 rng_state_0.pth
2025-08-18 18:25:19      16389 rng_state_1.pth
2025-08-18 18:25:19      16389 rng_state_2.pth
2025-08-18 18:25:20      16389 rng_state_3.pth
2025-08-18 18:25:20      16389 rng_state_4.pth
2025-08-18 18:25:20      16389 rng_state_5.pth
2025-08-18 18:25:20      16389 rng_state_6.pth
2025-08-18 18:25:21      16325 rng_state_7.pth
2025-08-18 18:25:21       1465 scheduler.pt
2025-08-18 18:25:21        613 special_tokens_map.json
2025-08-18 18:25:21  

Make the local directory

In [2]:
!mkdir -p ./rbio_TF_GO_EXP/
local_dir = './rbio_TF_GO_EXP/'

Model weights

In [3]:
!aws s3 cp s3://czi-rbio/rbio1-TF+GO+EXP/model.safetensors.index.json {local_dir} --no-sign-request
!aws s3 cp s3://czi-rbio/rbio1-TF+GO+EXP/model-00001-of-00002.safetensors {local_dir} --no-sign-request
!aws s3 cp s3://czi-rbio/rbio1-TF+GO+EXP/model-00002-of-00002.safetensors {local_dir} --no-sign-request

download: s3://czi-rbio/rbio1-TF+GO+EXP/model.safetensors.index.json to rbio_TF_GO_EXP/model.safetensors.index.json
download: s3://czi-rbio/rbio1-TF+GO+EXP/model-00001-of-00002.safetensors to rbio_TF_GO_EXP/model-00001-of-00002.safetensors
download: s3://czi-rbio/rbio1-TF+GO+EXP/model-00002-of-00002.safetensors to rbio_TF_GO_EXP/model-00002-of-00002.safetensors


Configuration and Tokenizer

In [4]:
!aws s3 cp s3://czi-rbio/rbio1-TF+GO+EXP/config.json {local_dir} --no-sign-request
!aws s3 cp s3://czi-rbio/rbio1-TF+GO+EXP/tokenizer.json {local_dir} --no-sign-request
!aws s3 cp s3://czi-rbio/rbio1-TF+GO+EXP/tokenizer_config.json {local_dir} --no-sign-request
!aws s3 cp s3://czi-rbio/rbio1-TF+GO+EXP/special_tokens_map.json {local_dir} --no-sign-request

download: s3://czi-rbio/rbio1-TF+GO+EXP/config.json to rbio_TF_GO_EXP/config.json
download: s3://czi-rbio/rbio1-TF+GO+EXP/tokenizer.json to rbio_TF_GO_EXP/tokenizer.json
download: s3://czi-rbio/rbio1-TF+GO+EXP/tokenizer_config.json to rbio_TF_GO_EXP/tokenizer_config.json
download: s3://czi-rbio/rbio1-TF+GO+EXP/special_tokens_map.json to rbio_TF_GO_EXP/special_tokens_map.json


Import libraries

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.modeling_utils import load_sharded_checkpoint
import re
import boto3
import os
import pandas as pd
from getpass import getpass
import torch
import warnings

warnings.filterwarnings('ignore')

pd.set_option('display.max_colwidth', None)

Define helper functions

In [21]:
def load_model_and_tokenizer(model_name, model_checkpoint_path, device):
    """Loads the base model and tokenizer, and applies the fine-tuned checkpoint."""
    # Load the base model from Hugging Face
    model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto")
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Load the fine-tuned weights from the local checkpoint directory
    if model_checkpoint_path is not None:
        print(f"Loading fine-tuned weights from {model_checkpoint_path}")
        load_sharded_checkpoint(model, model_checkpoint_path, strict=False)

    # Set tokenizer padding for batch processing
    tokenizer.padding_side = "left"
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    model.eval()
    model.to(device)
    return model, tokenizer


def ask_rbio(system_prompt, question, device, model, tokenizer):
    """Formats the prompt and generates a response from the model."""
    # Create the chat message format
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question},
    ]

    # Apply the chat template
    texts = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

    # Tokenize the input and move to the appropriate device (CPU/GPU)
    model_inputs = tokenizer(texts, return_tensors="pt", padding=True).to(device)

    # Generate a response
    generated_ids = model.generate(**model_inputs, max_new_tokens=1024, do_sample=False)
    generated_ids = [
        output_ids[len(input_ids) :]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    # Decode the response
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    # Print the raw response before parsing for debugging
    print("Raw model output:")
    print(response)
    print("-" * 20) # Separator for clarity


    # Parse the structured output using regex
    think_process = re.findall(r"<think>(.*?)</think>", response, flags=re.DOTALL)
    reflection = re.findall(r"<reflect>(.*?)</reflect>", response, flags=re.DOTALL)
    answer = re.findall(r"<answer>(.*?)</answer>", response, flags=re.DOTALL)

    # Return a dictionary with the parsed components
    parsed_response = {
        'think': think_process[0].strip() if think_process else "",
        'reflection': reflection[0].strip() if reflection else "",
        'answer': answer[0].strip() if answer else ""
    }
    return parsed_response

Download and load the model

In [7]:
!nvidia-smi

Wed Aug 27 21:19:44 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [8]:
os.listdir('/content/')

['.config', 'rbio_TF_GO_EXP', 'sample_data']

In [9]:
os.listdir('/content/rbio_TF_GO_EXP')

['config.json',
 'tokenizer.json',
 'model-00002-of-00002.safetensors',
 'model-00001-of-00002.safetensors',
 'tokenizer_config.json',
 'model.safetensors.index.json',
 'special_tokens_map.json']

In [10]:
# S3 bucket and model details
model_name = 'Qwen/Qwen2.5-3B-Instruct' # Base model from Hugging Face
model_checkpoint = 'rbio_TF_GO_EXP' # Fine-tuned checkpoint
local_dir = f'/content/{model_checkpoint}/' # Corrected local directory path

# Set device to GPU if available, otherwise CPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Load the model and tokenizer
model, tokenizer = load_model_and_tokenizer(model_name, local_dir, device)

Using device: cuda


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading fine-tuned weights from /content/rbio_TF_GO_EXP/


In [22]:
print(model)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-35): 36 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear(in_features=2048, out_features=256, bias=True)
          (v_proj): Linear(in_features=2048, out_features=256, bias=True)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (up_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Lin

In [26]:
system_prompt=( "A conversation between a User and a Biologist. The user asks a question, "
                  "and the Biologist solves it. The biologist first thinks about the reasoning process in the mind and "
                  "then provides the user with the answer. The reasoning process and answer are enclosed within "
                  "<think> </think> and <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> answer here </answer>. "
                  "The Biologist will evaluate each step of this problem, using logical reasoning and evidence from the prompt.")
question="Can you describe the roles of the V-type ATPase and proteins encoded by genes CHMP6, GGPS1, and RAB7A? Let's call these the Set1 proteins. Also describe the roles of the conserved oligomeric Golgi complex, the TRAPP complex, and the proteins encoded by genes CMAS, SEC22B, ST3GAL4, SYS1, WRB, YKT6, MGAT1, and SLC35A1. Let's call these Set2 proteins. Is there any reason why knockout of any protein in Set1 might have anticorrelated effects in the Golgi compared to knockout of any protein in Set2?"

In [27]:

response = ask_rbio(system_prompt, question, device, model, tokenizer)
print("Think Process:")
print(response['think'])
print("\nReflection:")
print(response['reflection'])
print("\nAnswer:")
print(response['answer'])

Raw model output:
<think> Knockout of any protein in Set1 (V-type ATPase and proteins encoded by genes CHMP6, GGPS1, and RAB7A) would likely affect the function of the Golgi apparatus. The V-type ATPase is a proton pump that is essential for maintaining the pH gradient across the Golgi membrane. CHMP6 is involved in the formation of multivesicular bodies, which can affect the transport vesicles. GGPS1 is involved in the synthesis of glycerophospholipids, which are important for the structure and function of the Golgi membrane. RAB7A is a small GTPase that regulates vesicle trafficking, and its absence could lead to defects in the Golgi. On the other hand, knockout of any protein in Set2 (conserved oligomeric Golgi complex, TRAPP complex, and proteins encoded by genes CMAS, SEC22B, ST3GAL4, SYS1, WRB, YKT6, MGAT1, and SLC35A1) would also affect the function of the Golgi apparatus. The conserved oligomeric Golgi complex is a set of proteins that form a scaffold for the Golgi apparatus, w

In [28]:
question2 = "Is there any reason why knockout of gene CHMP6 could have anticorrelated morphological effects in the Golgi compared to knockout of gene SLC35A1"

response2 = ask_rbio(system_prompt, question2, device, model, tokenizer)
print("Think Process:")
print(response2['think'])
print("\nReflection:")
print(response2['reflection'])
print("\nAnswer:")
print(response2['answer'])

Raw model output:
<think> Knockout of gene CHMP6 would likely lead to changes in the Golgi apparatus due to its role in membrane trafficking. SLC35A1 is involved in vesicle transport, which also affects the Golgi. However, the specific effects on the Golgi morphology might differ depending on the exact functions of these genes. Anticorrelated effects mean that the knockout of one gene results in opposite morphological changes compared to the other gene. </think> <answer> No, there is no inherent reason why the knockout of gene CHMP6 would have anticorrelated morphological effects in the Golgi compared to the knockout of gene SLC35A1. Both genes are involved in vesicle transport and membrane trafficking, but their specific roles and interactions with the Golgi apparatus might not result in opposite morphological changes. </answer>
--------------------
Think Process:
Knockout of gene CHMP6 would likely lead to changes in the Golgi apparatus due to its role in membrane trafficking. SLC35A

In [17]:
print(response2)

{'think': '', 'reflection': '', 'answer': ''}


In [25]:
question3 = "What is the role of gene CHMP6?"

response3 = ask_rbio(system_prompt, question3, device, model, tokenizer)
print("Think Process:")
print(response3['think'])
print("\nReflection:")
print(response3['reflection'])
print("\nAnswer:")
print(response3['answer'])

Raw model output:
Gene CHMP6, also known as Charge-Associated Membrane Protein 6, plays a significant role in cellular processes, particularly in the regulation of intracellular trafficking and vesicle formation. Here are some key points about its function:

1. **Vesicle Formation**: CHMP6 is involved in the formation of multivesicular bodies (MVBs), which are membrane-bound structures that contain both endosomal and lysosomal components. MVBs are crucial for the degradation of proteins and lipids that have reached their functional limits within the cell.

2. **Endosomal Sorting**: It is part of the ESCRT (Endosomal Sorting Complex Required for Transport) machinery, which is essential for sorting cargo proteins destined for degradation or recycling. This process involves the recognition of specific signals on the cargo proteins and the formation of vesicles to transport them to their final destinations.

3. **Regulation of Intracellular Trafficking**: CHMP6 helps regulate the trafficki